In [1]:
import GPSPreprocess as gps_preprocess
import ModeDetection as mode_detect
import Extractor
import route_solver as rs
import variable_generator
import activity_locations_identification as al_identifier
import Exceptions
import main as PyERT_Main

In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import fiona
import geojsonio as gjsio
import os
from pyrosm import OSM
import osmnx as ox
import networkx as nx
from geopy.geocoders import Nominatim
import math
from shapely.geometry import Point, LineString

Manual unit test for route_choice_gen function in route_solver module

In [3]:
# Collect trip segment and network data for test
test_trip_seg = gpd.read_file(filename= './test_data/test_rs_trip_seg')
network_g_from_xml = ox.graph_from_xml(filepath='./test_data/test_rs_network_g.osm', 
                                        bidirectional=False, simplify=False, retain_all=True)
network_g_from_xml = ox.projection.project_graph(network_g_from_xml)
network_n_from_xml, network_e_from_xml = ox.graph_to_gdfs(network_g_from_xml)

In [4]:
# Run route_choice_gen function to get output
test_trip_seg_route = rs.route_choice_gen(test_trip_seg, network_g_from_xml, network_e_from_xml, network_n_from_xml)

In [6]:
# Visualize original points in the trip segment for test
print(gjsio.make_url(test_trip_seg.to_json()))

http://geojson.io/#data=data:application/json,%7B%22type%22%3A%20%22FeatureCollection%22%2C%20%22features%22%3A%20%5B%7B%22id%22%3A%20%220%22%2C%20%22type%22%3A%20%22Feature%22%2C%20%22properties%22%3A%20%7B%22Modes%22%3A%20%22Dirve%22%2C%20%22RecordID%22%3A%20%2215325215%22%2C%20%22SerialID%22%3A%20%2214640_20070403_17%22%2C%20%22TimeStart%22%3A%20%2212%3A35%3A31%22%7D%2C%20%22geometry%22%3A%20%7B%22type%22%3A%20%22Point%22%2C%20%22coordinates%22%3A%20%5B-63.47442245%2C%2044.64709091%5D%7D%7D%2C%20%7B%22id%22%3A%20%221%22%2C%20%22type%22%3A%20%22Feature%22%2C%20%22properties%22%3A%20%7B%22Modes%22%3A%20%22Dirve%22%2C%20%22RecordID%22%3A%20%2215325218%22%2C%20%22SerialID%22%3A%20%2214640_20070403_17%22%2C%20%22TimeStart%22%3A%20%2212%3A36%3A11%22%7D%2C%20%22geometry%22%3A%20%7B%22type%22%3A%20%22Point%22%2C%20%22coordinates%22%3A%20%5B-63.47520065%2C%2044.64873123%5D%7D%7D%2C%20%7B%22id%22%3A%20%222%22%2C%20%22type%22%3A%20%22Feature%22%2C%20%22properties%22%3A%20%7B%22Modes%22%3A%20%2

In [7]:
# Visualize the route generated for the trip segment
print(gjsio.make_url(test_trip_seg_route[['SerialID','geometry']].to_json()))

http://geojson.io/#data=data:application/json,%7B%22type%22%3A%20%22FeatureCollection%22%2C%20%22features%22%3A%20%5B%7B%22id%22%3A%20%220%22%2C%20%22type%22%3A%20%22Feature%22%2C%20%22properties%22%3A%20%7B%22SerialID%22%3A%20%2214640_20070403_17%22%7D%2C%20%22geometry%22%3A%20%7B%22type%22%3A%20%22LineString%22%2C%20%22coordinates%22%3A%20%5B%5B-63.47455190000001%2C%2044.647136799999984%5D%2C%20%5B-63.4743594%2C%2044.64726909999999%5D%2C%20%5B-63.4742407%2C%2044.6473433%5D%2C%20%5B-63.47439500000001%2C%2044.64746119999999%5D%2C%20%5B-63.47500900000001%2C%2044.6478135%5D%2C%20%5B-63.4754972%2C%2044.647994499999996%5D%2C%20%5B-63.47590099999999%2C%2044.648327699999996%5D%2C%20%5B-63.475356299999994%2C%2044.6486729%5D%2C%20%5B-63.475225998790776%2C%2044.64875240150164%5D%2C%20%5B-63.4748303%2C%2044.6490106%5D%2C%20%5B-63.4747459%2C%2044.649085199999995%5D%2C%20%5B-63.474684%2C%2044.6491677%5D%2C%20%5B-63.47462709999999%2C%2044.6492602%5D%2C%20%5B-63.4745886%2C%2044.6493314%5D%2C%20%5B-6